In [ ]:
# =========== import modules ===========
import csv
import re
from datetime import datetime, timedelta
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import urllib
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
# ============= open the webpage ===============
url = "https://lweb.cfa.harvard.edu/shocks/wi_data/wi_1996.html"
page = urlopen(url)
html = page.read().decode("utf-8")

In [1]:
# ========= find the year =============
soup = BeautifulSoup(html, 'html.parser')

# find the text in the last row of the table
all_row = soup.table.find_all('tr')
last_row = all_row[-1]

# get the year from the text
year = int(last_row.td.text.strip().split()[-1])

NameError: name 'BeautifulSoup' is not defined

In [ ]:
# ======== Find the type cell FF =========
def Find_FF(row):
    global Type
    soup = BeautifulSoup(str(row), 'html.parser')

    # Check if the 'td' tag containing the text 'SF' is present
    for td in soup.find_all('td'):
        if re.search(r'\s*FF\s*', td.get_text()):
            Type = 'FF'
            break
    else:
        Type = 'other'

In [ ]:
# =========== Find DOY ===========
def Find_DOY(row):
    global value
    soup = BeautifulSoup(str(row), 'html.parser')

    # Find the table row (<tr>) you want to extract from
    tr = soup.find('tr')

    # Find the specific table data cell (<td>) you want to extract the value from
    td = tr.find_all('td')[3]

    # Extract the text value from the table data cell
    value = str(td.get_text())

In [ ]:
# ============= If type is FF record DOY ===============
doy = []
date_time = []

for i in range(7,len(all_row)):
    row = all_row[i]
    Find_FF(row)
    Find_DOY(row)
    if Type == 'FF':
        # ============= convert DOY into DD/MM/YYY  HH:MM:SS ========================
        days = float(value)
        conversion = datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1)
        # format: YYYY/MM/DD  HH:MM:SS
        doy.append(value)
        date_time.append(conversion)

In [ ]:
dates_date = np.array([[date.year, date.month, date.day] for date in date_time])
dates_time = np.array([[date.hour, date.minute, date.second] for date in date_time])

Date = np.array([dates_date,dates_time][0])
Time = np.array([dates_date,dates_time][1])

In [ ]:
row = []
row = [np.transpose(doy), Date, Time]

In [ ]:
# ============ write in the csv file ===============
csv_filename = 'data.csv'
csv_file = open(csv_filename, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['DOY', 'Year','Month','Day', 'Hour', 'Minutes', 'Second'])
for j in range(0, len(row[0])):
    csv_writer.writerow([row[0][j], row[1][j][0], row[1][j][1],row[1][j][2], row[2][j][0], row[2][j][1], row[2][j][2]])

csv_file.close()